In [15]:
import pandas as pd
import itertools
import collections
from pandas.core.frame import DataFrame
import numpy as np
import os
import openpyxl
import re
import wordcloud
import jieba
import jieba.analyse# TF-IDF的关键词的抽取
import jieba.posseg
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = ['sans-serif'] 
plt.rcParams['font.sans-serif'] = ['SimHei']

In [22]:
import mlxtend
from mlxtend.frequent_patterns import apriori  #挖掘频繁项集
from mlxtend.frequent_patterns import association_rules #挖掘关联规则

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator
from collections import Counter

In [ ]:
#引用中文字体
WordCloud(font_path='C:\Windows\Fonts\HGDBS_CNKI.TTF')

· 读取数据转文本

In [ ]:
data = pd.read_excel(r'E:\Ajiao\LiXian\notebook\notebook\事故调查报告列表-中远海重.xls',usecols=[49])
data

# 1.去除虚词和停用词

In [ ]:
def stopwordslist():
    stopwordslist =[line.strip() for line in open('D:\AAA安装\jieba-0.42.1\jieba\dict\虚词和停用词.txt',encoding='UTF-8').readlines()]
    return stopwordslist

In [ ]:
#seg_depart：源列表
#stopwordslist:停用词列表
def seg_depart(sen):
    sen_depart=jieba.cut(sen.strip())
    stopwords = stopwordslist()
    outstr = ''
    for word in sen_depart:
        if word not in stopwords:
            if word !='\t':
                outstr += word
                outstr += ''
    return outstr

In [ ]:
outfilename = 'D:\AAA安装\jieba-0.42.1\jieba\dict\out.txt'
outputs = open(outfilename,'w',encoding='UTF-8')
# 转文本[data['详细描述'].iloc[i] for i in range(len(data)) if i<=len(data)]

for i in [data['详细描述'].iloc[i] for i in range(len(data)) if i<=len(data)]:
    empty_word=seg_depart(i)
    outputs.write(empty_word+'\n')
outputs.close()
print('删除虚词和停用词成功！！！')

# 2.读取文件

In [ ]:
f = open('D:\AAA安装\jieba-0.42.1\jieba\dict\out.txt','r',encoding='UTF-8')
data_mix = f.read()
print(data_mix)

# 3.归并词群表

In [ ]:
result_mix_1=re.sub(r'(\u8131\u79bb)|(\u6ed1\u8131)|(\u8131\u5f00)','脱落',data_mix)
result_mix_2=re.sub(r'(\u5207\u5272)|(\u4fee\u5272)|(\u5272\u65ad)|(\u5272\u5f00)|(\u5272\u9664)|(\u5207\u9664)|(\u5272\u7834)','割伤',result_mix_1)
result_mix_3=re.sub(r'(\u4e0b\u5760)|(\u6eda\u843d)','坠落',result_mix_2)
result_mix_4=re.sub(r'(\u7838\u4e0b)|(\u7838\u5230)','砸伤',result_mix_3)
result_mix_5=re.sub(r'\u7ef7\u65ad','崩脱',result_mix_4)
result_mix_6=re.sub(r'(\u71c3\u7206)|(\u7206\u70b8)|(\u7206\u71c3)|(\u5f15\u71c3)|(\u95ea\u7206)|(\u95ea\u71c3)','爆裂',result_mix_5)
result_mix_7=re.sub(r'(\u4e0b\u6ed1)|(\u6ed1\u5165)|(\u6ed1\u52a8)','滑落',result_mix_6)
result_mix_8=re.sub(r'(\u6324\u538b)','挤伤',result_mix_7)
result_mix_9=re.sub(r'(\u7ede\u5165)','绞伤',result_mix_8)
result_mix_10=re.sub(r'(\u649e\u5230)|(\u649e\u51fb)|(\u78b0\u649e)|(\u78b0\u843d)','撞倒',result_mix_9)
result_mix_11=re.sub(r'(\u8e29\u5230)|(\u8e29\u7a7a)|(\u8e29\u788e)','踩踏',result_mix_10)
result_mix_12=re.sub(r'(\u503e\u659c)','倾倒',result_mix_11)
result_mix_13=re.sub(r'(\u6d82\u88c5)','涂装车间',result_mix_12)#涂装转为涂装车间，以免提取成特征项

# 4.去除时间和英文字母和换行符和年月日

In [ ]:
result = re.sub(r'(\d+|[a-zA-Z]+|\n+|[\u5e74-\u6708-\u65e5])','',result_mix_13)
result

# 5.分词

In [ ]:
data_lcut=jieba.lcut(result,cut_all=False)
data_lcut

# 6.专业术语词库

In [ ]:
dict_1= 'D:\AAA安装\jieba-0.42.1\jieba\dict\专业术语词汇.txt'
jieba.load_userdict(dict_1)
f = open(dict_1,'r',encoding='UTF-8')
for i in f.readlines():
    jieba.suggest_freq(i,tune=True)

In [ ]:
j =jieba.lcut(result)
j
#result 因为运行过上个词典了

# 7.去无关词   

In [ ]:
file_1=open('D:\AAA安装\jieba-0.42.1\jieba\dict\去无关词.txt',encoding='UTF-8').read()
file = jieba.lcut(file_1)

In [ ]:
for i in file:
    if i in jieba.lcut(result):
        result = result.replace(i,'')  
result

# 8.提取特征项

In [ ]:
r = jieba.analyse.extract_tags(result,topK=30,allowPOS=('v','x','p','r','u','c','e','a','d','f','g'))
trait='/'.join(r)
trait

In [ ]:
r

# 9.帕累托图

In [ ]:
cou = []
word_list=jieba.lcut(result)
for i in range(len(r)):
    if r[i] in word_list:
        cou.append(word_list.count(r[i]))
        i=i+1
cou

In [ ]:
data_trai=DataFrame({'频数':cou},index=r)
da_trait=data_trai.sort_values(by=['频数'],ascending=False)
da_trait

In [ ]:
da_trait['Characteristic factors']=['Ac1','Ac2','Ac3','Ac4','Ac5','Ac6','Ac7','Ac8','Ac9','Ac10','Ac11','Ac12','Ac13','Ac14','Ac15','Ac16','Ac17','Ac18','Ac19','Ac20','Ac21','Ac22','Ac23','Ac24','Ac25','Ac26','Ac27','Ac28','Ac29','Ac30']
da_trait.index = da_trait['Characteristic factors']
da_trait

In [ ]:
p = da_trait['频数'].cumsum()/da_trait['频数'].sum()  #计算累计营收比
key = p[p>0.8].index[0]  #标记80%，节点索引
key_num = da_trait.index.tolist().index(key)#索引位置

da_trait['频数'].plot(kind = 'bar', color = '#48D1CC', edgecolor = 'black',alpha = 0.6, width = 0.6,rot=0)
plt.ylabel('Frequency')
plt.xlabel('Characteristic factors')

plt.xticks(fontsize=8)

p.plot(style = '--k.',secondary_y = True)
plt.axvline(key_num,color='r',linestyle="--",alpha=0.8)  
plt.text(key_num+0.2,p[key],'The cumulative proportion is：%.3f%%' % (p[key]*100), color = 'r')  # 累计占比超过80%的节点
plt.ylabel('Frequency cumulative percentage')
#这里根据特征因素的数量进行边距的设置
plt.xlim(-0.5,len(da_trait['频数'])-0.5)
plt.grid(alpha=0.5,linestyle='--')

# 10 绘制词云图

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
words = [
    ("cut", 558),
    ("come down", 500),
    ("dump", 283),
    ("slide", 278),
    ("fall off", 257),
    ("bruise", 128),
    ("instability", 112),
    ("ballast", 103),
    ("clear", 100),
    ("burst", 99),
    ("trample", 83),
    ("bump", 64),
    ("hoist", 56),
    ("repair", 51),
    ("fasten", 51),
    ("burn", 50),
    ("avoid", 47),
    ("smash", 46),
    ("slip", 44),
    ("wound", 40),
    ("fall", 40),
    ("lighter", 40),
    ("scald", 39),
    ("lifting", 39),
    ("pump pressure", 36),
    ("sucker", 36),
    ("transport", 35),
    ("scratch", 35),
    ("laceration", 31),
    ("hook", 11),
]
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="WordCloud-shape-diamond"))
    .render(r"E:\基于文本挖掘的船舶建造事故致因分析\文本挖掘论文\英文附件\词云图英文\wordcloud_diamond.html")
)

# 11.提取部门    

In [ ]:
apart=jieba.analyse.extract_tags(result,topK=13,withWeight=False,allowPOS=('nn',))#从文本中提取部门
#提取的是出现的频率最高的前几个
apart

# 12 归并词群表 提取部门与特征项

In [ ]:
for i in range(len(data)):
    #将涂装xx转为涂装车间，方便提取部门，而涂装在事故报告中不一定是事故发生在涂装车间，还有可能是在其他车间涂装队的某某参与了工作
    if ('涂装科' in data['详细描述'].iloc[i])|('涂装1科' in data['详细描述'].iloc[i])|('涂装2科' in data['详细描述'].iloc[i])|( '舾装工区' in data['详细描述'].iloc[i])|( '舾装部' in data['详细描述'].iloc[i])|('服务工区' in data['详细描述'].iloc[i])|('机电工区' in data['详细描述'].iloc[i]):
        data['详细描述'].iloc[i]=re.sub(r'(\u6d82\u88c5\u79d1)','涂装车间',data['详细描述'].iloc[i])
        data['详细描述'].iloc[i]=re.sub(r'(\u6d82\u88c5\d\u79d1)','涂装车间',data['详细描述'].iloc[i])  
        data['详细描述'].iloc[i]=re.sub(r'(\u6d82\u88c5\d\u79d1)','涂装车间',data['详细描述'].iloc[i])        
        data['详细描述'].iloc[i]=re.sub(r'(\u823e\u88c5\u5de5\u533a)','舾装车间',data['详细描述'].iloc[i])
        data['详细描述'].iloc[i]=re.sub(r'(\u823e\u88c5\u90e8)','舾装车间',data['详细描述'].iloc[i])
        data['详细描述'].iloc[i]=re.sub(r'(\u670d\u52a1\u5de5\u533a)','服务车间',data['详细描述'].iloc[i])
        data['详细描述'].iloc[i]=re.sub(r'(\u673a\u7535\u5de5\u533a)','机电车间',data['详细描述'].iloc[i])
data
#没有将 舾装部机装科或舾装部船装科或舾装一体 合并维舾装车间；只是对详细描述进行分析
#将舾装部、舾装工区合并为舾装车间
#服务工区转为服务车间
#机电工区  机电车间

In [ ]:
apart_update=['船体车间', '涂装车间', '机电车间', '服务车间', '起运车间', '船台工区', '舾装车间']

In [ ]:
for i in range(len(data)):
    data['详细描述'].iloc[i]=re.sub(r'(\u4e0b\u5760)|(\u6eda\u843d)','坠落',data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u8131\u79bb)|(\u6ed1\u8131)|(\u8131\u5f00)','脱落',data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u7838\u4e0b)|(\u7838\u5230)','砸伤', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u5207\u5272)|(\u4fee\u5272)|(\u5272\u65ad)|(\u5272\u5f00)|(\u5272\u9664)|(\u5207\u9664)|(\u5272\u7834)','割伤', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'\u7ef7\u65ad','崩脱', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u71c3\u7206)|(\u7206\u70b8)|(\u7206\u71c3)|(\u5f15\u71c3)|(\u95ea\u7206)|(\u95ea\u71c3)','爆裂', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u4e0b\u6ed1)|(\u6ed1\u5165)|(\u6ed1\u52a8)','滑落', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u6324\u538b)','挤伤', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u7ede\u5165)','绞伤', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u649e\u5230)|(\u649e\u51fb)|(\u78b0\u649e)|(\u78b0\u843d)','撞倒', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u8e29\u5230)|(\u8e29\u7a7a)|(\u8e29\u788e)','踩踏', data['详细描述'].iloc[i])
    data['详细描述'].iloc[i]=re.sub(r'(\u503e\u659c)','倾倒', data['详细描述'].iloc[i])
#     上一个语句已经改为涂装车间，不需要进行归并了
#     data['详细描述'].iloc[i]=re.sub(r'(\u6d82\u88c5\u79d1)','涂装车间', data['详细描述'].iloc[i])
data

# 13 提取部门、特征项

In [ ]:
data['department']=''
data['trait']=''
data 

In [ ]:
def extract_r(s,k): #s为apart或者r  k为1或2
    for j in range(len(data)):
        for i in range(len(s)):
            if s[i] in str(data['详细描述'][j]):
                w = data.iloc[:,k][j]
                data.iloc[:,k][j] = str(w)+','+s[i]
    return data

In [ ]:
extract_r(apart,1)
extract_r(r,2)

In [ ]:
data[5:10]

In [ ]:
#去除前面的逗号
data['department']=data['department'].apply(lambda x:x.strip(','))
data['trait'] =data['trait'].apply(lambda x:x.strip(','))
data

# 14 验证部门与特征项的的正确与否

In [ ]:
data['test_1']=False
data['test_2']=False
data

In [ ]:
def test():
    for k in range(1,3):
        for i in range(len(data)):
            if ',' in data.iloc[:,k][i]:
                b = data.iloc[:,k][i].split(',')
                for j in range(len(b)):
                    if b[j] in data['详细描述'][i]:
                        data.iloc[:,k+2][i]=True
                    elif not b[j] in data['详细描述'][i]:
                        data.iloc[:,k+2][i]=False
                        break
            else:
                if data.iloc[:,k][i] in data['详细描述'][i]:
                    data.iloc[:,k+2][i]=True
    return data

In [ ]:
test()

In [ ]:
data['test_1'].value_counts()
data['test_2'].value_counts()

In [ ]:
# 14 部门与特征项的关联关系

### 三、导入数据

In [ ]:
for k in range(1,3): 
    for i in range(len(data)):
        if len(data.iloc[:,k][i])==0:
            data.iloc[:,k][i]=np.nan
dat=data.iloc[:,:3]
dat

In [ ]:
#nan 的数量
count=0
for i in range(len(data)):
    if dat['trait'][i] is np.nan:
        count = count+1
count

### 15 特征项的关联

In [ ]:
dat_trait =dat.dropna(subset=['trait'])
dat_trait

In [ ]:
lst = dat_trait['trait'].apply(lambda x:x.split(',')).tolist()
lst

In [ ]:
with open('demo.txt','w',encoding = 'utf-8') as  f:
    f.write(str(lst))

### 16 部门与特征项之间的原始数据

In [ ]:
dat_department = dat.dropna()
dat_depart = dat_department[['department','trait']]
dat_depart

In [ ]:
#候选项集
d_t = dat_depart.apply(lambda x:x['department'] + ',' + x['trait'],axis=1).apply(lambda x:x.split(','))
lst_apart = d_t.tolist()
lst_apart# 原始数据集

In [ ]:
len(lst_apart)

## 17 绘制雷达图

In [ ]:
def dataset(depart):
    l = []  #车间
    ss=[]
    for key in lst_apart:
        if depart in key:
            l.append(key)
    for value in r:       #致因项
        con = 0     
        for key in l:     #车间
            if value in key: 
                s = 1
                con = con + s
            else:
                s = 0
        percent = con/len(l)
        pd.DataFrame()
        ss.append([value,depart,con,percent])
    return ss

In [ ]:
data_jidian = pd.DataFrame(dataset('机电车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_jidian.head(8)

In [ ]:
jidian_eng=data_jidian.head(8)
jidian_english=jidian_eng.replace({'坠落':'come down','滑落':'slide','割伤':'cut','砸伤':'bruise','爆裂':'burst','压载':'ballast','倾倒':'dump','修理':'repair'})
jidian_english

In [ ]:
data_chuanti =pd.DataFrame(dataset('船体车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_chuanti.head(8)

In [ ]:
chuanti_eng=data_chuanti.head(8)
chuanti_english=chuanti_eng.replace({'坠落':'come down','滑落':'slide','割伤':'cut','砸伤':'bruise','脱落':'fall off','踩踏':'trample','倾倒':'dump','磕碰':'bump'})
chuanti_english

In [ ]:
data_fuwu =pd.DataFrame(dataset('服务车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_fuwu.head(8)

In [ ]:
fuwu_eng=data_fuwu.head(8)
fuwu_english=fuwu_eng.replace({'坠落':'come down','滑落':'slide','割伤':'cut','起钩':'hook','脱落':'fall off','吊起':'hoist','倾倒':'dump','起吊':'lifting'})
fuwu_english

In [ ]:
data_qiyun =pd.DataFrame(dataset('起运车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_qiyun.head(8)

In [ ]:
qiyun_eng=data_qiyun.head(8)
qiyun_english=qiyun_eng.replace({'坠落':'come down','滑落':'slide','起钩':'hook','失稳':'instability','脱落':'fall off','避让':'avoid','倾倒':'dump','吊起':'hoist'})
qiyun_english

In [ ]:
data_xizhuang = pd.DataFrame(dataset('舾装车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_xizhuang.head(8)

In [ ]:
xizhuang_eng=data_xizhuang.head(8)
xizhuang_english=xizhuang_eng.replace({'坠落':'come down','滑落':'slide','割伤':'cut','砸伤':'bruise','脱落':'fall off','失稳':'instability','倾倒':'dump','磕碰':'bump'})
xizhuang_english

In [ ]:
data_chuantai = pd.DataFrame(dataset('船台工区'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_chuantai.head(8)

In [ ]:
chuantai_eng=data_chuantai.head(8)
chuantai_english=chuantai_eng.replace({'坠落':'come down','滑落':'slide','割伤':'cut','砸伤':'bruise','清理':'repair','吊起':'hoist','倾倒':'dump','起吊':'lifting'})
chuantai_english

In [ ]:
data_tuzhuang = pd.DataFrame(dataset('涂装车间'),columns=['致因项','部门','次数','百分比']).sort_values(by=['次数'],ascending=False)
data_tuzhuang.head(8)

In [ ]:
tuzhuang_eng=data_tuzhuang.head(8)
tuzhuang_english=tuzhuang_eng.replace({'坠落':'come down','击伤':'wound','割伤':'cut','裂伤':'laceration','脱落':'fall off','失稳':'instability','倾倒':'dump','爆裂':'burst'})
tuzhuang_english

In [ ]:
plt.rcParams['font.sans-serif'] = 'SimHei'

fig = plt.figure(figsize=(15,6),dpi=500)
ax1 = fig.add_subplot(241,polar=True)
ax1.set_title('Hull workshop')
angles_1 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_1 = np.concatenate((angles_1,[angles_1[0]])) 
values = data_chuanti['百分比'].head(8)
feature = chuanti_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax1.plot(angles_1, values, '*-', linewidth=2, linestyle='--',markersize=10) #线条
ax1.fill(angles_1, values, alpha=0.25)  #填充颜色

# 标签
ax1.set_thetagrids(angles_1 * 180/np.pi, feature, fontproperties = "SimHei")
ax1.grid(True)


ax2=fig.add_subplot(242, polar=True)
ax2.set_title('Electromechanical workshop')
angles_2 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_2 = np.concatenate((angles_2,[angles_2[0]])) 
values = data_jidian['百分比'].head(8)
feature = jidian_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax2.plot(angles_2, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax2.fill(angles_2, values, alpha=0.25)
# 标签
ax2.set_thetagrids(angles_2 * 180/np.pi, feature, fontproperties = "SimHei")
ax2.grid(True)

ax3=fig.add_subplot(243, polar=True)
ax3.set_title('Shipping workshop')
angles_3 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_3 = np.concatenate((angles_3,[angles_3[0]])) 
values = data_qiyun['百分比'].head(8)
feature = qiyun_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax3.plot(angles_3, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax3.fill(angles_3, values, alpha=0.25)
# 标签
ax3.set_thetagrids(angles_3 * 180/np.pi, feature, fontproperties = "SimHei")
ax3.grid(True)

ax4=fig.add_subplot(244, polar=True)
ax4.set_title('Service workshop')
angles_4 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_4 = np.concatenate((angles_4,[angles_4[0]])) 
values = data_fuwu['百分比'].head(8)
feature = fuwu_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax4.plot(angles_4, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax4.fill(angles_4, values, alpha = 0.25)
# 标签
ax4.set_thetagrids(angles_4 * 180/np.pi, feature, fontproperties = "SimHei")
ax4.grid(True)

ax5=fig.add_subplot(245, polar=True)
ax5.set_title('Painting workshop')
angles_5 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_5 = np.concatenate((angles_5,[angles_5[0]])) 
values = data_tuzhuang['百分比'].head(8)
feature = tuzhuang_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax5.plot(angles_5, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax5.fill(angles_5, values, alpha=0.25)
# 标签
ax5.set_thetagrids(angles_5 * 180/np.pi, feature, fontproperties = "SimHei")
ax5.grid(True)

ax6=fig.add_subplot(246, polar=True)
ax6.set_title('Outfitting workshop')
angles_6 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_6 = np.concatenate((angles_6,[angles_6[0]])) 
values = data_xizhuang['百分比'].head(8)
feature = xizhuang_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax6.plot(angles_6, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax6.fill(angles_6, values, alpha=0.25)
# 标签
ax6.set_thetagrids(angles_6 * 180/np.pi, feature, fontproperties = "SimHei")
ax6.grid(True)

ax7=fig.add_subplot(247, polar=True)
ax7.set_title('Slipway workshop')
fig.tight_layout()
angles_7 = np.linspace(0, 2*np.pi,8, endpoint=False)
angles_7 = np.concatenate((angles_7,[angles_7[0]])) 
values = data_chuantai['百分比'].head(8)
feature = chuantai_english['致因项'].head(8)
values = np.concatenate((values,[values.iloc[0]]))    # 数值,百分比
feature = np.concatenate((feature,[feature.iloc[0]])) # 事故致因项
ax7.plot(angles_7, values, '*-', linewidth=2,linestyle='--',markersize=10)
ax7.fill(angles_7, values, alpha=0.25)
# 标签
ax7.set_thetagrids(angles_7 * 180/np.pi, feature, fontproperties = "SimHei")
ax7.grid(True)

### 一、部门与特征之间的联系

In [ ]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet: #数据集中每一条事务
        for item in transaction: #事务中的每一个项集
            if not {item} in C1: #判断每一个项集在不在候选集合中
                C1.append({item})#添加不在候选集中的那些项集
                C1.sort() #排序
    return list(map(frozenset, C1))

In [ ]:
def scanD(D, Ck, minSupport):
    ssCnt = {}          #字典的形式存放项集及其出现次数，{项集:次数}
    for tid in D:       #每一条事务集
        for can in Ck:  #候选集中每一个项集
            if can.issubset(tid): #判断can是否是tid的子集，返回的是布尔型数据
                ssCnt[can] = ssCnt.get(can, 0) + 1 
    numItems = float(len(D))         #数据集长度
    retList= []                      #频繁项集
    supportData = {} #候选集项Ck的支持度字典(key:候选项， value:支持度)
    for key in ssCnt:
        support = ssCnt[key] / numItems #计算每一项集的支持度
        supportData[key] = support #将所有项集及其支持度以字典的形式放入supportData
        if support >= minSupport: #如果支持度满足最小支持度要求（≥最小支持度）
            retList.append(key) #把满足支持度的项集放入频繁项集retList     
    return retList, supportData 

In [ ]:
def aprioriGen(Lk):
    Ck = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = Lk[i]
            L2 = Lk[j]
            C = Lk[i] | Lk[j]
            if not C in Ck and (len(C)==len(Lk[0])+1):
                Ck.append(Lk[i] | Lk[j])
    return Ck

In [ ]:
def apriori(D, minSupport):
    C1 = createC1(D) #生成候选1项集
    L1, supportData = scanD(D, C1, minSupport) #生成频繁1项集和支持度列表
    L = [L1] #将频繁1项集放入L中
    k=2 #项集数，初始值设为2
    while (len(L[-1]) > 0): #如果L中最后项集中的元素数不为0则持续循环
        Ck = aprioriGen(L[-1]) #生成候选k项集
        Lk, supK = scanD(D, Ck, minSupport) #根据候选k项集，生成频繁k项集和支持度列表
        supportData.update(supK) #更新支持度列表
        L.append(Lk) #更新频繁项集L
        k=k+1
    return L, supportData

In [ ]:
#二项集
def calcConf(freqSet, H, supportData, brl, minConf):
    prunedH = [] #放置置信度过滤后的可以出现在规则右部的元素列表
    for conseq in H: #对关联规则右部元素列表进行循环
        conf = supportData[freqSet]/supportData[freqSet-conseq] #计算规则的置信度
        lift = conf/supportData[conseq]
        counti = supportData[freqSet]*len(lst_apart) #出现次数
        if conf >= minConf: #判断置信度是否满足最小置信度的要求
             for i in apart_update:   #去掉致因项之间的关联
                if (i in conseq) | (i in freqSet-conseq): # 前者或后者有部门
                    print(freqSet-conseq,'-->',conseq,'support_data:',round(supportData[freqSet],3),'conf:',round(conf,3),'lift:',round(lift,3),'count:',round(counti,3)) #打印强关联规则
                    brl.append((freqSet-conseq, conseq, supportData[freqSet],conf,lift,counti)) #把强关联规则追加到 brl
                    prunedH.append(conseq) #将经过置信度过滤的元素，放入prunedH
    return prunedH 

In [ ]:
#多项集
def rulesFromConseq(freqSet, H, supportData, brl, minConf):
    Hmp = True
    while Hmp:
        Hmp = False
        H = calcConf(freqSet, H, supportData, brl, minConf)
        H = aprioriGen(H)
        Hmp = not(H == [] or len(H[0]) == len(freqSet))

In [ ]:
def generateRules(L, supportData, minConf):
    bigRuleList = [] #强关联规则容器
    for i in range(1, len(L)): #对所有频繁项集循环操作，这里需要注意的是频繁1项集不存在关联规则
        for freqSet in L[i]: # 对频繁项集中的每个子集关联规则进行挖掘
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList,minConf) #频繁多项集
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf) #频繁二项集
    return bigRuleList

In [ ]:
#支持度、频繁项集
L,suppData=apriori(lst_apart,minSupport=0.01)
suppData

In [ ]:
#置信度
cc = generateRules(L,suppData,minConf=0.1)
len(cc)

In [ ]:
#转变冷冻集合
def filter_data(s):
    import re
    match = re.search(r"('(.*)')" , s).group()
    if match:
        s = eval(match)
    return s

In [ ]:
related_items=pd.DataFrame(cc,columns=['Source','Target','support_data','conf','lift','count'])
related_items['Source']=related_items['Source'].astype(str)
related_items['Target']=related_items['Target'].astype(str)
related_items['Target'] = related_items['Target'].apply(filter_data)
related_items['Source'] = related_items['Source'].apply(filter_data)
related_items

In [ ]:
related_items.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\Relate.csv',encoding='utf_8_sig')
#部门与事故致因项置信度写进excel中

In [ ]:
apart_trait=pd.DataFrame(r).rename(columns={0:'trait'})
for i in range(len(apart_update)):
    apart_trait.loc[30+i] = apart_update[i]
apart_trait

In [ ]:
apart_trait.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\apart_trait.csv',encoding="utf_8_sig")
# 所有部门与特征项

## 卡方检验：部门与致因项

In [ ]:
# 双向无序R*C卡方检验：得到的p值是整体P值，若p<0.05,则这几组中有一组与其他组不同，但是具体哪一组并不知道，需要进行两两检验。
# 两两检验的其中一种：bonferroni校正法

In [ ]:
# dd = pd.read_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\Relate_SPSS.csv',encoding="utf_8_sig")
# dd

In [ ]:
# dd['count'] = ''
# for j in range(len(dd)):
#     con = 0
#     for value in lst_apart:
#         if len(dd['Source'].iloc[j])!=12:
#             if (dd['Source'].iloc[j] in value) & (dd['Target'].iloc[j] in value):
#                 con = con + 1
#         else:
#             ddc = re.findall(r"([\u4e00-\u9fa5]{2})",dd['Source'].iloc[j])
#             if (ddc[0] in value) & (dd['Target'].iloc[j] in value) & (ddc[1] in value):
#                 con = con + 1
#     dd['count'].iloc[j] = con
# dd

In [ ]:
dd.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\Related_spss.csv',encoding="utf_8_sig")

In [ ]:
# 可以在related_items中直接删除重复的值，得到的就是频数可以直接用

#### 计算致因项在部门的百分比 : 用雷达图计算百分比，根据关联规则算的总数不对

In [ ]:
# dd['Source'] = dd['Source'].apply(lambda x:x.split(','))
# dd

In [ ]:
# dd2 = dd.explode('Source').reset_index().drop(columns=['index'])
# dd2

In [ ]:
# for i in range(len(dd2)):
#     if "'" in dd2['Source'].iloc[i]:
#         dd2['Source'].iloc[i]= re.search(r"([\u4e00-\u9fa5]{2})",dd2['Source'].iloc[i]).group()
# dd2

In [ ]:
# dd3 = dd2[['Source','Target','count']]
# per = dd3.groupby(['Target','Source']).agg({'count':'sum'})
# per.reset_index()

In [ ]:
#这样算是多加了的
# su = per.reset_index().groupby('Target')['count'].sum().to_frame()
# su
#对原始数据dd求部门和
# su = dd.groupby(['Target'])['count'].sum().to_frame()
# su

In [ ]:
# pert = pd.merge(per.reset_index(),su,on = 'Target')
# pert['percent']  = pert['count_x']/pert['count_y']
# pert

In [ ]:
# pert.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\percent.csv',encoding='utf_8_sig')

### 散点图：支持度+提升度

In [ ]:
#事故发生部门与事故致因项之间
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

fig = plt.figure(figsize=(10,6),dpi=500)
x = related_items['support_data'].values
y = related_items['lift'].values

cm = plt.cm.get_cmap('RdYlBu')

z = related_items['conf'].values

sc = plt.scatter(x,y,c=z,cmap=cm)
plt.colorbar(sc,label='Confidence')

plt.xlabel('Support')
plt.ylabel('Lift')

# plt.title('事故部门与致因项')
plt.show()

#### 二、事故致因之间的联系

In [ ]:
lst

In [ ]:
def createC1(dataSet):
    C1 = [[i] for i in np.unique(np.concatenate(lst).ravel())]
    return C1

In [ ]:
base = createC1(lst)
base

In [ ]:
#判断是否满足子集的条件,
#找出所有子集
def inn(ls,items):
    N = len(items)
    flag = 0
    for i in range(2 ** N):
        combo = []
        for j in range(N):
            if (i >> j) % 2 :
                combo.append(items[j])
        if ls == combo:
            flag += 1
        else:
            flag += 0
    if flag > 0:
        return True
    else:
        return False

In [ ]:
#s所有子集列表
s=[]
for q in lst:
    N = len(q)
    for i in range(2**N):
        combo = []
        for j in range(N):
            if (i>>j)%2:
                combo.append(q[j])
                s.append(combo)
#列表去重之后，原始数据集中的所有子集
com=[]
for i in s:
    if i not in com:
        com.append(i)
com

In [ ]:
#列表去重之后，原始数据集中的所有子集
com=[]
for i in s:
    if i not in com:
        com.append(i)
com

In [ ]:
#计算子集出现的频数
def scan(lst,com): 
    dic={}
    for i in com:
        for j in lst:
            if inn(i,j):
                dic[str(i)] = dic.get(str(i),0) + 1
    return dic

In [ ]:
dic={}
for i in com:
    for j in lst:
        if inn(i,j):
            dic[str(i)] = dic.get(str(i),0) + 1
dic

In [ ]:
#产生庞大的数据集，把没有出现过的数据集删除
def get_support(lst,min_support):
    dic = {}
    dic.update(scan(lst,com))
    num = len(lst)    
    support_dic = {}
    count = 0
    for key,value in dic.items():
        support = value/num
        if support >= min_support:
            support_dic[key] =support
            count += 1 
    return support_dic

In [ ]:
reason_su=get_support(lst,0.005)
reason_su# 频繁项集

In [ ]:
def get_rlue(support_dic):
    count = 0
    rulelist = []
    for key,value in support_dic.items():#支持度频繁项集
        n = len(eval(key))
        if  n == 2 :
            front = eval(key)[0]
            after = eval(key)[1]
            confi = support_dic[key]/support_dic[str([front])]
            lifti = confi/support_dic[str([after])]
            counti = support_dic[key]*len(lst)  #次数
            print(f'{[front]}','-->',f'{[after]}','\n',#打印所有前键和后键
                     'together support:',round(support_dic[key],3),',',#联合支持度也就是联合概率
                     'conf:',round(confi,3),',',
                     'lift:',round(lifti,3),','
                     'count:',round(counti,3))
            rulelist.append((front,after,round(support_dic[key],3),round(confi,3),round(lifti,3),round(counti,3)))
        elif n > 2:
            front = eval(key)[:n-1]
            after = eval(key)[-1]
            confi = support_dic[key]/support_dic[str(front)]
            lifti = confi/support_dic[str([after])]
            counti =  support_dic[key]*len(lst)
            print(f'{[front]}','-->',f'{[after]}','\n',
             'support:',round(support_dic[key],3),',',#联合支持度也就是联合概率
             'conf:',round(confi,3),',',
             'lift:',round(lifti,3),','
             'count:',round(counti,3))
            rulelist.append((front,after,round(support_dic[key],3),round(confi,3),round(lifti,3),round(counti,3)))
        count += 1
    return rulelist

In [ ]:
reason=get_rlue(get_support(lst,0.005))
reason   

In [ ]:
cc_11=pd.DataFrame(reason,columns=['Source','Target','support','conf','lift','count'])
cc_11

In [ ]:
cc_11.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\trai_Related.csv',encoding='utf_8_sig')

In [ ]:
trait_xls=pd.DataFrame(base)
trait_xls

In [ ]:
trait_xls.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\trai.csv',encoding='UTF_8_sig')

In [ ]:
# 事故致因项之间
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

fig = plt.figure(figsize=(10,6),dpi=500)
x = cc_11['support'].values
y = cc_11['lift'].values

cm = plt.cm.get_cmap('RdYlBu')

z = cc_11['conf'].values

sc = plt.scatter(x,y,c=z,cmap=cm)
plt.colorbar(sc,label='Confidence')

plt.xlabel('Support')
plt.ylabel('Lift')

# plt.title('事故部门与致因项')
plt.show()

## 卡方检验：事故致因项（有序）

In [ ]:
# 双向有序的R*C列表: 要用非参数检验 + 两两检验

In [ ]:
# 有序的先计算频数,事故致因项是有序的，从原始数据中找出所有子集以及所有子集的频数，
# 事故发生部门与事故致因项之间的频数是正确的，支持度*原始数据总长度=频数
# 存在的多个致因项与车间部门之间的关联关系频数也可以这样算

In [ ]:
cc_count = cc_11[['Source','Target','count']]
cc_count

In [ ]:
cc_count.to_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\trait_Spss.csv',encoding='UTF_8_sig')